In [16]:
import os
import cv2
import random
import shutil
import torchvision.transforms as transforms


from xeception_2x1ch import *
from xeception import *
from googlenetv4 import *
import pandas as pd
import numpy as np

In [29]:
radius_list = []
def concat_origin_img(white_img, black_img):
    white_img = 255 - white_img

    img_sum = np.zeros((white_img.shape[0],white_img.shape[1],3), dtype=np.uint8)
    img_sum[:,:,0] = white_img
    img_sum[:,:,1] = black_img
    
    return img_sum

def find_circles(img_origin):
    global radius_list
    try:
        
        img = cv2.cvtColor(img_origin, cv2.COLOR_BGR2GRAY)
        #가우시안필터
        gaus = cv2.GaussianBlur(img, (5, 5), 7)

        #적응형이진화
        thres = cv2.adaptiveThreshold(gaus, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 5)
        thres = cv2.GaussianBlur(thres, (5, 5), 7)
        
        circles = cv2.HoughCircles(thres, cv2.HOUGH_GRADIENT, 1, 1200, param1 = 250, param2 = 60, minRadius = 720, maxRadius = 800)
        
        if len(circles) > 0:
            for i in circles[0]:
                cx = i[0]
                cy = i[1]
                radius = i[2]
                radius_list.append(radius)
        else:
            print("cannot find circles")
        #이미지 ROI 추출
        margin = 40 # 여백
        s_radius = 740
        x_min = int(cx - s_radius - margin)
        y_min = int(cy - s_radius - margin)
        x_max = int(cx + s_radius + margin)
        y_max = int(cy + s_radius + margin)
        img_roi = img_origin[y_min:y_max,x_min:x_max]
        img_roi = cv2.resize(img_roi,(1500,1500))
        # show_subplot_img([img_roi], 15)
        
        return True, img_roi
    
    except Exception as e:
        print(e)
        return False, img

In [32]:
testimg_base_dir = 'E:/test_temp/'
target_dir = '/test'
dir_list = os.listdir(testimg_base_dir)

for dirs in dir_list:
    os.makedirs(testimg_base_dir+dirs+target_dir, exist_ok=True)
    img_list = [x for x in os.listdir(testimg_base_dir+dirs) if x.endswith(('jpg', 'bmp'))]
    for files in img_list:
        file_key = '_'.join(files.split("_")[:-1])
        contains_string = df['id'].str.contains(file_key).any()
        if contains_string:
            continue
        else:
            try:
                w_name = file_key+'_0.jpg'
                b_name = file_key+'_1.jpg'
                w_img = cv2.imread(testimg_base_dir+dirs +'/'+ w_name , 0)
                b_img = cv2.imread(testimg_base_dir+dirs +'/'+ b_name , 0)
                img_sum = concat_origin_img(w_img, b_img)

                ref, img_roi = find_circles(img_sum)
                if ref:
                    cv2.imwrite(testimg_base_dir+dirs+target_dir + '/' + b_name, img_roi)
                else:
                    continue
            except:
                continue

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of t

In [28]:
sum(radius_list) / len(radius_list)

741.4494524826812

In [4]:
import test
import numpy as np
import time

base_path = r'E:\test_temp\for_testing'

for dirs in os.listdir(base_path):
    print(dirs)
    start_time = time.time()
    img_path = base_path + '/' + dirs
    out = test.classfication(img_path=img_path)

    results_array = np.array(out)

    # count number of True values per column
    column_counts = np.sum(results_array, axis=0)

    # count number of True values in total
    total_count = len(out)

    print("Count per column:", column_counts)
    print("Total count:", total_count)
    print("total time : ", time.time() - start_time)
    print("average time : ", (time.time() - start_time) / total_count)

broken
E:\test_temp\for_testing/broken is a directory.
E:\test_temp\for_testing/broken is a directory.
Count per column: [ 16  31   0 103   4   1   0]
Total count: 269
total time :  18.929418325424194
average time :  0.07036958485287804
burr
E:\test_temp\for_testing/burr is a directory.
E:\test_temp\for_testing/burr is a directory.
Count per column: [  0 116 569  46   8  25   0]
Total count: 668
total time :  43.626259088516235
average time :  0.06530877109059317
b_bubble
E:\test_temp\for_testing/b_bubble is a directory.
E:\test_temp\for_testing/b_bubble is a directory.
Count per column: [ 18  11   0   3 737  27   0]
Total count: 840
total time :  54.057318687438965
average time :  0.06435395081837972
b_edge
E:\test_temp\for_testing/b_edge is a directory.
E:\test_temp\for_testing/b_edge is a directory.
Count per column: [ 54 136   0   1   0   2   0]
Total count: 279
total time :  19.071402311325073
average time :  0.06835628068575295


In [1]:
import test
import numpy as np
import time

base_path = r'E:\test_temp\for_testing'

for dirs in os.listdir(base_path):
    print(dirs)
    start_time = time.time()
    img_path = base_path + '/' + dirs
    out = test.classfication(img_path=img_path,selected_model='googlenetv4')

    results_array = np.array(out)

    # count number of True values per column
    column_counts = np.sum(results_array, axis=0)

    # count number of True values in total
    total_count = len(out)

    print("Count per column:", column_counts)
    print("Total count:", total_count)
    print("total time : ", time.time() - start_time)
    print("average time : ", (time.time() - start_time) / total_count)

d:\SHkim\contactlensCC_wxpython\wx_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


broken
E:\test_temp\for_testing/broken is a directory.
E:\test_temp\for_testing/broken is a directory.
Count per column: [  4  17   0 236   4   1   0]
Total count: 269
total time :  15.34300947189331
average time :  0.05703720993268889
burr
E:\test_temp\for_testing/burr is a directory.
E:\test_temp\for_testing/burr is a directory.
Count per column: [  4  49 646  12   1  11   0]
Total count: 668
total time :  29.403756141662598
average time :  0.04401759901446497
b_bubble
E:\test_temp\for_testing/b_bubble is a directory.
E:\test_temp\for_testing/b_bubble is a directory.
Count per column: [ 38   8   8  45 772   9   0]
Total count: 840
total time :  35.81145620346069
average time :  0.042632685956500826
b_edge
E:\test_temp\for_testing/b_edge is a directory.
E:\test_temp\for_testing/b_edge is a directory.
Count per column: [ 66 206   0   7   0   5   0]
Total count: 279
total time :  12.623862266540527
average time :  0.045246818159643466
